This notebook contains an example for teaching.

## Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question,
but we could begin to investigate from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender. Two main questions here are:


* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data


The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015.  We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors;  individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$. 

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size $n=5150$.

## Data analysis

We start by loading the data set.

In [1]:
load("../data/wage2015_subsample_inference.Rdata")
dim(data)

[1] 5150   20

Let's have a look at the structure of the data.

In [2]:
str(data)
data1 <- data

'data.frame':	5150 obs. of  20 variables:
 $ wage : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage: num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex  : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg  : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg  : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2 : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3 : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4 : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 269 23 99 86 226 232 184 146 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 19 1 6 5 17 17 13 10 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 12 165 231 176 171 135 210 201 ...
 $ ind2 : Factor w/ 

### Cleaning data

Focus on people who did not go to college. The variable `shs (Some High School)` and `hsg (High School Graduate)` help us to identify people that did not go to college.

In [3]:
data = data1[ data1$shs == 1 | data1$hsg == 1, ]

We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [4]:
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]

cat("Number of observation:", n, '\n')
cat( "Number of raw regressors:", p)

Number of observation: 1376 
Number of raw regressors: 18

For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [5]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")

Warning message:
"package 'xtable' was built under R version 4.1.0"


In [6]:
library(xtable)
Z_subset <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"))]
table <- matrix(0, 12, 1)
table[1:12,1]   <- as.numeric(lapply(Z_subset,mean))
rownames(table) <- c("Log Wage","Sex","Some High School","High School Graduate","Some College","College Graduate", "Advanced Degree","Midwest","South","West","Northeast","Experience")
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

,Sample mean
,<dbl>
Log Wage,2.7185624
Sex,0.3219477
Some High School,0.0872093
High School Graduate,0.9127907
Some College,0.0000000
College Graduate,0.0000000
Advanced Degree,0.0000000
Midwest,0.2863372
South,0.2914244


E.g., the share of female workers in our sample is ~44% ($sex=1$ if female).

Alternatively, we can also print the table as latex.

In [7]:
print(tab, type="latex")

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% 
\begin{tabular}{rr}
  \hline
 & Sample mean \\ 
  \hline
Log Wage & 2.72 \\ 
  Sex & 0.32 \\ 
  Some High School & 0.09 \\ 
  High School Graduate & 0.91 \\ 
  Some College & 0.00 \\ 
  College Graduate & 0.00 \\ 
  Advanced Degree & 0.00 \\ 
  Midwest & 0.29 \\ 
  South & 0.29 \\ 
  West & 0.20 \\ 
  Northeast & 0.22 \\ 
  Experience & 17.19 \\ 
   \hline
\end{tabular}


## Prediction Question

Now, we will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

\begin{equation}\label{decompose}
Y = \beta'X+ \epsilon.
\end{equation}

Our goals are

* Predict wages  using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators, regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is *experience* times the indicator of having a *college degree*.

Using the **Flexible Model**, enables us to approximate the real relationship by a
 more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [8]:
# 1. basic model
basic <- lwage~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2)
regbasic <- lm(basic, data=data)
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model



Call:
lm(formula = basic, data = data)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
  2.8330066   -0.0733094    0.0075742   -0.0811342           NA           NA  
        clg           mw           so           we        occ22        occ23  
         NA   -0.0431882   -0.1091620    0.0129620   -0.1961261   -0.0086113  
      occ24        occ25        occ26        occ27        occ28        occ29  
  0.0005078    0.2615289   -0.3510072   -0.1900342   -0.6616521   -0.3013316  
     occ210       occ211       occ212       occ213       occ214       occ215  
 -0.0576220   -0.4176903   -0.4663571   -0.4219896   -0.5527766   -0.4747648  
     occ216       occ217       occ218       occ219       occ220       occ221  
 -0.2381724   -0.3529422   -0.3976108   -0.1181885   -0.1053967   -0.1737437  
     occ222        ind23        ind24        ind25        ind26        ind27  
 -0.3479965    0.1742747    0.0504201    0.0585330    0.0348081    0.2379530

Number of regressors in the basic model: 51 


The `lm` function drops three variables hsg, scl and clg. We have to take this into account since Python does not drop these variables because it does not use the QR decomposition as R does. We shall have to drop the variables manually.

##### Note that the basic model consists of $48$ regressors. Three regressors were dropped out.

In [9]:
# 2. flexible model
flex <- lwage ~ sex + (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2
regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length( regflex$coef ) ) # number of regressors in the Flexible Model


Call:
lm(formula = flex, data = data)

Coefficients:
  (Intercept)            sex           exp1           exp2           exp3  
    1.616e+01     -8.833e-02     -3.404e+00      3.827e+01     -2.073e+01  
         exp4            shs            hsg            scl            clg  
    6.983e+00     -6.386e-01             NA             NA             NA  
        occ22          occ23          occ24          occ25          occ26  
   -2.543e+00     -4.076e+01      7.848e+00     -5.326e+01      2.041e+01  
        occ27          occ28          occ29         occ210         occ211  
    1.894e+00     -1.016e+01     -2.071e+01     -3.932e+00      3.123e-01  
       occ212         occ213         occ214         occ215         occ216  
    1.845e+00     -2.104e+00     -1.179e+00     -1.081e+01     -1.757e-01  
       occ217         occ218         occ219         occ220         occ221  
   -4.729e+00     -3.364e-01     -6.760e+00     -2.967e+00     -1.216e+00  
       occ222          ind23      

Number of regressors in the flexible model: 980

### Selecting column names that are not NAN

In [10]:
names_variables_na_flex <- attributes(regflex$coef)$names[ !is.na( regflex$coef ) ]
names_variables_na_flex

df_flex <- model.matrix(regflex)

clean_data_flex <- df_flex[ , names_variables_na_flex ]

[1] "(Intercept)"   "sex"           "exp1"          "exp2"         
  [5] "exp3"          "exp4"          "shs"           "occ22"        
  [9] "occ23"         "occ24"         "occ25"         "occ26"        
 [13] "occ27"         "occ28"         "occ29"         "occ210"       
 [17] "occ211"        "occ212"        "occ213"        "occ214"       
 [21] "occ215"        "occ216"        "occ217"        "occ218"       
 [25] "occ219"        "occ220"        "occ221"        "occ222"       
 [29] "ind23"         "ind24"         "ind25"         "ind26"        
 [33] "ind27"         "ind28"         "ind29"         "ind210"       
 [37] "ind211"        "ind212"        "ind213"        "ind214"       
 [41] "ind215"        "ind216"        "ind217"        "ind218"       
 [45] "ind219"        "ind220"        "ind221"        "ind222"       
 [49] "mw"            "so"            "we"            "exp1:exp4"    
 [53] "exp1:shs"      "exp1:occ22"    "exp1:occ23"    "exp1:occ24"   
 [57] "exp1:occ25"    "exp1:occ26"    "exp1:occ27"    "exp1:occ28"   
 [61] "exp1:occ29"    "exp1:occ210"   "exp1:occ211"   "exp1:occ212"  
 [65] "exp1:occ213"   "exp1:occ214"   "exp1:occ215"   "exp1:occ216"  
 [69] "exp1:occ217"   "exp1:occ218"   "exp1:occ219"   "exp1:occ220"  
 [73] "exp1:occ221"   "exp1:occ222"   "exp1:ind23"    "exp1:ind24"   
 [77] "exp1:ind25"    "exp1:ind26"    "exp1:ind27"    "exp1:ind28"   
 [81] "exp1:ind29"    "exp1:ind210"   "exp1:ind211"   "exp1:ind212"  
 [85] "exp1:ind213"   "exp1:ind214"   "exp1:ind215"   "exp1:ind216"  
 [89] "exp1:ind217"   "exp1:ind218"   "exp1:ind219"   "exp1:ind220"  
 [93] "exp1:ind221"   "exp1:ind222"   "exp1:mw"       "exp1:so"      
 [97] "exp1:we"       "exp2:exp4"     "exp2:shs"      "exp2:occ22"   
[101] "exp2:occ23"    "exp2:occ24"    "exp2:occ25"    "exp2:occ26"   
[105] "exp2:occ27"    "exp2:occ28"    "exp2:occ29"    "exp2:occ210"  
[109] "exp2:occ211"   "exp2:occ212"   "exp2:occ213"   "exp2:occ214"  
[113] "exp2:occ215"   "exp2:occ216"   "exp2:occ217"   "exp2:occ219"  
[117] "exp2:occ220"   "exp2:occ221"   "exp2:occ222"   "exp2:ind23"   
[121] "exp2:ind24"    "exp2:ind25"    "exp2:ind26"    "exp2:ind27"   
[125] "exp2:ind28"    "exp2:ind29"    "exp2:ind210"   "exp2:ind211"  
[129] "exp2:ind212"   "exp2:ind213"   "exp2:ind214"   "exp2:ind215"  
[133] "exp2:ind216"   "exp2:ind217"   "exp2:ind218"   "exp2:ind219"  
[137] "exp2:ind220"   "exp2:ind221"   "exp2:ind222"   "exp2:mw"      
[141] "exp2:so"       "exp2:we"       "exp3:exp4"     "exp3:shs"     
[145] "exp3:occ22"    "exp3:occ23"    "exp3:occ24"    "exp3:occ26"   
[149] "exp3:occ27"    "exp3:occ28"    "exp3:occ29"    "exp3:occ210"  
[153] "exp3:occ211"   "exp3:occ212"   "exp3:occ213"   "exp3:occ214"  
[157] "exp3:occ215"   "exp3:occ216"   "exp3:occ217"   "exp3:occ219"  
[161] "exp3:occ220"   "exp3:occ221"   "exp3:occ222"   "exp3:ind23"   
[165] "exp3:ind24"    "exp3:ind25"    "exp3:ind26"    "exp3:ind27"   
[169] "exp3:ind28"    "exp3:ind29"    "exp3:ind210"   "exp3:ind211"  
[173] "exp3:ind212"   "exp3:ind213"   "exp3:ind214"   "exp3:ind216"  
[177] "exp3:ind217"   "exp3:ind218"   "exp3:ind219"   "exp3:ind220"  
[181] "exp3:ind221"   "exp3:ind222"   "exp3:mw"       "exp3:so"      
[185] "exp3:we"       "exp4:shs"      "exp4:occ22"    "exp4:occ23"   
[189] "exp4:occ24"    "exp4:occ26"    "exp4:occ28"    "exp4:occ29"   
[193] "exp4:occ210"   "exp4:occ211"   "exp4:occ212"   "exp4:occ213"  
[197] "exp4:occ214"   "exp4:occ215"   "exp4:occ216"   "exp4:occ217"  
[201] "exp4:occ219"   "exp4:occ220"   "exp4:occ221"   "exp4:occ222"  
[205] "exp4:ind23"    "exp4:ind24"    "exp4:ind25"    "exp4:ind26"   
[209] "exp4:ind27"    "exp4:ind28"    "exp4:ind29"    "exp4:ind210"  
[213] "exp4:ind211"   "exp4:ind212"   "exp4:ind213"   "exp4:ind214"  
[217] "exp4:ind216"   "exp4:ind217"   "exp4:ind218"   "exp4:ind219"  
[221] "exp4:ind220"   "exp4:ind221"   "exp4:ind222"   "exp4:mw"      
[225] "exp4:so"       "exp4:we"       "shs:occ24"     "shs:occ211"   
[229] "shs:o

##### Save data

In [11]:
save(clean_data_flex, file = "../data/wg2_clean_data_flex.RData")

save(df_flex, file = "../data/wg2_data_flex.RData")

Note that the flexible model consists of $494$ regressors of $979$ variables.

Try Lasso next

In [12]:
library(hdm)
flex <- lwage ~ sex + (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2
lassoreg<- rlasso(flex, data=data)

sumlasso<- summary(lassoreg)


Warning message:
"package 'hdm' was built under R version 4.1.0"



Call:
rlasso.formula(formula = flex, data = data)

Post-Lasso Estimation:  TRUE 

Total number of variables: 979
Number of selected variables: 12 

Residuals: 
      Min        1Q    Median        3Q       Max 
-1.585175 -0.304508 -0.006954  0.274110  3.558977 

              Estimate
(Intercept)      2.627
sex              0.000
exp1             0.005
exp2             0.000
exp3             0.000
exp4             0.000
shs              0.000
hsg              0.000
scl              0.000
clg              0.000
occ22            0.000
occ23            0.000
occ24            0.000
occ25            0.000
occ26            0.000
occ27            0.000
occ28            0.000
occ29            0.000
occ210           0.000
occ211           0.000
occ212           0.000
occ213          -0.225
occ214          -0.270
occ215           0.000
occ216           0.000
occ217           0.000
occ218           0.000
occ219           0.000
occ220           0.000
occ221           0.000
occ222           0.000


In [13]:
length(sumlasso$coef)

[1] 980

In [14]:
var_dif_zero <- attributes(sumlasso$coef)$names[ sumlasso$coef > 0 ]
var_dif_zero

[1] "(Intercept)"   "exp1"          "exp1:hsg"      "occ214:ind25" 
[5] "occ214:ind28"  "occ22:ind210"  "occ220:ind211" "occ215:ind213"

Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [15]:
# Assess the predictive performance

sumbasic <- summary(regbasic)
sumflex <- summary(regflex)

#  R-squared 
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
R2.adj1 <- sumbasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adj1, "\n")

R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
R2.adj2 <- sumflex$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adj2, "\n")

R2.L <- sumlasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.L, "\n")
R2.adjL <- sumlasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjL, "\n")

R-squared for the basic model:  0.1802381 
adjusted R-squared for the basic model:  0.1512255 
R-squared for the flexible model:  0.5099981 
adjusted R-squared for the flexible model:  0.2352412 
R-squared for the lasso with flexible model:  0.08936084 
adjusted R-squared for the flexible model:  0.08134348 


In [16]:
# calculating the MSE
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")
p1 <- sumbasic$df[1] # number of regressors
MSE.adj1 <- (n/(n-p1))*MSE1
cat("adjusted MSE for the basic model: ", MSE.adj1, "\n")

MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")
p2 <- sumflex$df[1]
MSE.adj2 <- (n/(n-p2))*MSE2
cat("adjusted MSE for the flexible model: ", MSE.adj2, "\n")


MSEL <-mean(sumlasso$res^2)
cat("MSE for the lasso flexible model: ", MSEL, "\n")
pL <- length(sumlasso$coef)
MSE.adjL <- (n/(n-pL))*MSEL
cat("adjusted MSE for the lasso flexible model: ", MSE.adjL, "\n")

MSE for the basic model:  0.2082191 
adjusted MSE for the basic model:  0.2157451 
MSE for the flexible model:  0.1244602 
adjusted MSE for the flexible model:  0.1943896 
MSE for the lasso flexible model:  0.2313019 
adjusted MSE for the lasso flexible model:  0.8037156 


In [17]:
model.matrix(sumlasso)

,sex,exp1,exp2,exp3,exp4,shs,hsg,scl,clg,occ22,...,ind216:we,ind217:we,ind218:we,ind219:we,ind220:we,ind221:we,ind222:we,mw:so,mw:we,so:we
15,0,18,3.24,5.832,10.4976,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43,1,42,17.64,74.088,311.1696,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,0,37,13.69,50.653,187.4161,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,1,31,9.61,29.791,92.3521,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,1,7,0.49,0.343,0.2401,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,0,30,9.00,27.000,81.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
113,0,16,2.56,4.096,6.5536,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,0,27,7.29,19.683,53.1441,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128,0,6,0.36,0.216,0.1296,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129,0,8,0.64,0.512,0.4096,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
library(xtable)
table <- matrix(0, 3, 5)
table[1,1:5]   <- c(p1,R2.1,MSE1,R2.adj1,MSE.adj1)
table[2,1:5]   <- c(p2,R2.2,MSE2,R2.adj2,MSE.adj2)
table[3,1:5]   <- c(pL,R2.L,MSEL,R2.adjL,MSE.adjL)
colnames(table)<- c("p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$")
rownames(table)<- c("basic reg","flexible reg", "lasso flex")
tab<- xtable(table, digits =c(0,0,2,2,2,2))
print(tab,type="latex") # type="latex" for printing table in LaTeX
tab

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% 
\begin{tabular}{rrrrrr}
  \hline
 & p & \$R\verb|^|2\_\{sample\}\$ & \$MSE\_\{sample\}\$ & \$R\verb|^|2\_\{adjusted\}\$ & \$MSE\_\{adjusted\}\$ \\ 
  \hline
basic reg & 48 & 0.18 & 0.21 & 0.15 & 0.22 \\ 
  flexible reg & 495 & 0.51 & 0.12 & 0.24 & 0.19 \\ 
  lasso flex & 980 & 0.09 & 0.23 & 0.08 & 0.80 \\ 
   \hline
\end{tabular}


,p,$R^2_{sample}$,$MSE_{sample}$,$R^2_{adjusted}$,$MSE_{adjusted}$
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
basic reg,48,0.18023815,0.2082191,0.15122549,0.2157451
flexible reg,495,0.50999815,0.1244602,0.23524115,0.1943896
lasso flex,980,0.08936084,0.2313019,0.08134348,0.8037156


Considering all measures above, the flexible model performs slightly better than the basic model. 

One procedure to circumvent this issue is to use **data splitting** that is described and applied in the following.

## Data Splitting

Measure the prediction quality of the two models via data splitting:

- Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophiscticated version of splitting that we can consider).
- Use the training sample for estimating the parameters of the Basic Model and the Flexible Model.
- Use the testing sample for evaluation. Predict the $\mathtt{wage}$  of every observation in the testing sample based on the estimated parameters in the training sample.
- Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models. 

In [19]:
dim(data)

[1] 1376   20

In [20]:
#splitting the data

set.seed(1) # to make the results replicable (generating random numbers)
random <- sample(1:n, floor(n*4/5))
# draw (4/5)*n random numbers from 1 to n without replacing them
train <- data[random,] # training sample
test <- data[-random,] # testing sample
print(dim(train))
print(dim(test))

[1] 1100   20
[1] 276  20


In [21]:
# basic model
# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)
regbasic

# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)
trainregbasic

y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)

cat("Test MSE for the basic model: ", MSE.test1, " ")

cat("Test R2 for the basic model: ", R2.test1)


Call:
lm(formula = basic, data = train)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   2.836027    -0.113776     0.008171    -0.045630           NA           NA  
        clg           mw           so           we        occ22        occ23  
         NA    -0.007887    -0.092965     0.027930    -0.198916    -0.014664  
      occ24        occ25        occ26        occ27        occ28        occ29  
   0.202306     0.242367     0.006158    -0.180164    -0.796769    -0.241348  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.015354    -0.399688    -0.506190    -0.446041    -0.575386    -0.450461  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.209308    -0.338949    -0.251586    -0.137541    -0.124913    -0.158960  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.350055     0.191507     0.036101     0.034002    -0.004881     0.21484

Warning message in predict.lm(regbasic, newdata = test):
"prediction from a rank-deficient fit may be misleading"


43       47      158      248      261      482      858      936 
2.872798 2.475568 2.909436 2.843801 2.706183 2.816292 3.030303 2.580716 
     954     1323     1490     1567     1630     1870     1964     2025 
2.801527 2.787998 3.062171 2.836898 2.326333 2.979703 2.285942 2.276830 
    2063     2362     2674     2925     3021     3065     3084     3170 
2.412655 2.711248 2.784423 2.636268 2.744333 2.677121 2.840804 3.074304 
    3385     3395     3581     3639     3715     3759     3770     3797 
2.828920 3.123457 2.388790 2.799951 3.210152 2.792775 2.264362 2.738699 
    3876     3917     4182     4183     4198     4222     4417     4610 
2.615367 2.817868 2.850550 2.798970 2.905476 2.129962 2.997718 2.490984 
    4671     4809     4869     4940     5069     5073     5180     5224 
2.297529 2.774117 2.733286 2.812450 2.890822 2.827126 2.760961 2.889827 
    5344     5598     5801     5848     6122     6236     6314     6533 
2.848974 3.119391 2.354723 2.940427 2.762220 2.724100 2.860109 2.901859 
    6844     6874     6881     7034     7137     7171     7309     7322 
2.265131 2.460612 2.263418 3.180935 2.501392 2.390470 2.795458 2.496359 
    7335     7381     7389     7929     7998     8136     8350     8471 
2.809982 2.997905 2.803136 2.738447 2.526222 2.960109 2.769129 2.708267 
    8478     8517     8572     8574     8578     8647     8802     8880 
2.664675 2.806314 2.703490 2.958472 2.820964 2.734791 2.906452 2.785470 
    9002     9111     9154     9360     9432     9456     9469     9488 
2.464904 2.993126 2.318446 2.779223 2.921167 2.636552 2.930964 2.704377 
    9504     9731     9770     9771     9852     9980    10208    10445 
2.421110 2.575738 2.505990 2.752206 2.809982 2.695625 2.825308 2.378906 
   10464    10709    10797    10905    11286    11490    11526    11542 
2.793640 2.795029 2.962288 2.826964 2.841088 2.933928 2.747153 2.839341 
   11858    12271    12361    12418    12439    12731    12753    12764 
3.083966 2.808405 2.480362 2.686605 2.844052 2.793640 2.986799 2.733305 
   12786    12867    12895    12980    13040    13105    13221    13354 
2.569409 2.760837 2.705152 2.948344 2.730090 2.968491 2.502339 2.901246 
   13358    13421    13603    13616    13728    13767    13858    13859 
2.482359 3.125308 2.160298 2.746005 2.666227 2.802424 2.752024 3.040440 
   14106    14134    14141    14189    14252    14326    14337    14683 
2.722042 2.407710 2.389074 2.669847 2.595581 2.843988 2.675921 2.824746 
   14692    14705    14748    14919    14943    14956    15015    15071 
2.876767 2.885334 2.712918 2.734495 2.450420 2.782129 2.572474 2.693666 
   15105    15193    15253    15295    15299    15385    15508    16328 
2.440899 2.458114 3.381992 2.756010 2.720160 3.187121 2.502988 2.643000 
   16365    16523    16841    17079    17152    17177    17249    17402 
2.535670 2.380429 2.893551 2.893823 2.723327 2.757586 2.886939 2.268831 
   18230    18257    18336    18375    18528    18558    18700    18846 
2.326511 2.729321 2.714885 2.658776 2.715157 2.387997 2.578446 2.620074 
   19056    19152    19181    19263    19615    19848    19987    19995 
2.845393 2.988053 2.934211 2.734891 2.547116 2.409090 2.303548 2.521727 
   20049    20062    20116    20198    20410    20423    20456    20653 
2.788741 2.619299 2.747839 2.702963 2.651368 2.899300 2.253588 2.704025 
   20777    20916    21030    21543    21626    21929    22080    22721 
2.479894 2.623468 2.858322 3.296119 2.892065 2.588540 2.680694 2.521727 
   22734    23053    23146    23353    23405    23517    23848    23991 
2.690374 2.718013 2.293828 2.776616 2.815392 2.704888 2.670543 2.800256 
   24374    24514    24649    25233    25417    25519    25767    25804 
3.132884 2.424292 2.569726 2.293828 1.983952 2.414019 2.866345 2.841260 
   25937    26067    26135    26331    26426    26620    26623    26696 
3.049481 2.589698 2.917757 3.106391 2.950831 2.512371 2.744083 2.750562 
   26804    26936    27200    27379    27395    27491    27541  

Test MSE for the basic model:  0.2015247  Test R2 for the basic model:  0.06784807

In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$.

In [22]:
# flexible model
# estimating the parameters
#options(warn=-1)
regflex <- lm(flex, data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test)
y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- ( MSE.test2/var(y.test) )

cat("Test MSE for the flexible model: ", MSE.test2, "\n")

cat("Test R2 for the flexible model: ", R2.test2)

Warning message in predict.lm(regflex, newdata = test):
"prediction from a rank-deficient fit may be misleading"


Test MSE for the flexible model:  23072.14 
Test R2 for the flexible model:  -106719.1

In the flexible model, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is not large.

It is worth to notice that the $MSE_{test}$ vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs is about as well (or slightly better) than the flexible model. 


Next, let us use lasso regression in the flexible model instead of ols regression. Lasso (*least absolute shrinkage and selection operator*) is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors $p$ is relatively large in relation to $n$. 

Note that the out-of-sample $MSE$ on the test sample can be computed for any other black-box prediction method as well. Thus, let us finally compare the performance of lasso regression in the flexible model to ols regression.

In [23]:
# flexible model using lasso

# estimating the parameters
library(hdm)
reglasso <- rlasso(flex, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglasso<- predict(reglasso, newdata=test)
MSE.lasso <- sum((y.test-trainreglasso)^2)/length(y.test)
R2.lasso<- 1- MSE.lasso/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE.lasso, " ")

cat("Test R2 for the lasso flexible model: ", R2.lasso)

Test MSE for the lasso on flexible model:  0.2045988  Test R2 for the lasso flexible model:  0.05362881

Finally, let us summarize the results:

In [24]:
table2 <- matrix(0, 3,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[3,1]   <- MSE.lasso
table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2
table2[3,2]   <- R2.lasso

rownames(table2)<- c("basic reg","flexible reg","lasso regression")
colnames(table2)<- c("$MSE_{test}$", "$R^2_{test}$")
tab2 <- xtable(table2, digits =3)
tab2

,$MSE_{test}$,$R^2_{test}$
,<dbl>,<dbl>
basic reg,2.015247e-01,6.784807e-02
flexible reg,2.307214e+04,-1.067191e+05
lasso regression,2.045988e-01,5.362881e-02


In [25]:
print(tab2,type="latex") # type="latex" for printing table in LaTeX

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% 
\begin{tabular}{rrr}
  \hline
 & \$MSE\_\{test\}\$ & \$R\verb|^|2\_\{test\}\$ \\ 
  \hline
basic reg & 0.202 & 0.068 \\ 
  flexible reg & 23072.138 & -106719.127 \\ 
  lasso regression & 0.205 & 0.054 \\ 
   \hline
\end{tabular}


# Partialling Out

In [26]:
basic1 = lwage ~ exp1 + shs + hsg + scl + clg + mw + so + we + occ2 + ind2
basic2 = sex ~ exp1 + shs + hsg + scl + clg + mw + so + we + occ2 + ind2

rl <- rlasso(basic1, data= data , post=TRUE)$res
rs <- rlasso(basic2, data= data , post=TRUE)$res

basic_partial.est <- summary(lm( rl ~ rs ))$coef[2,1]
cat("Coefficient for SEX via partialling-out",basic_partial.est)

Coefficient for SEX via partialling-out -0.09065628

In [27]:
flex1 = lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2
flex2 = sex ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2

rl <- rlasso(flex1, data= data , post=TRUE)$res
rs <- rlasso(flex2, data= data , post=TRUE)$res

flex_partial.est <- summary(lm( rl ~ rs ))$coef[2,1]
cat("Coefficient for SEX via flex partialling-out",flex_partial.est)

Coefficient for SEX via flex partialling-out -0.08141371